In [1]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

In [3]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        # take each word and tokenize it
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # adding documents
        documents.append((w, intent['tag']))

        # adding classes to our class list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [4]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print (len(documents), "documents")

print (len(classes), "classes", classes)

print (len(words), "unique lemmatized words", words)


pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

65 documents
21 classes ['anxiety', 'behavioral_activation', 'cognitive_restructuring', 'coping_with_triggers', 'depression', 'exposure_therapy', 'goal_setting', 'goodbye', 'gratitude', 'greeting', 'introduction', 'mindfulness_relaxation', 'negative_thoughts', 'positive_affirmations', 'positive_thinking', 'problem_solving', 'progress_tracking', 'relationships', 'self_esteem', 'stress', 'thought_therapists']
106 unique lemmatized words ['a', 'about', 'achieving', 'activation', 'affirmation', 'anxiety', 'anxious', 'appreciate', 'are', 'attack', 'behavioral', 'breakup', 'bye', 'can', 'cbt', 'challenge', 'changing', 'cognitive', 'communication', 'confidence', 'cope', 'day', 'deal', 'do', 'doe', 'down', 'dream', 'empty', 'enough', 'exercise', 'explain', 'exposure', 'feeling', 'for', 'give', 'goal', 'good', 'goodbye', 'guide', 'handling', 'hello', 'help', 'hey', 'hi', 'hopeless', 'how', 'i', 'improvement', 'in', 'increase', 'is', 'issue', 'it', 'low', 'manage', 'me', 'measure', 'mindfulness'

In [5]:
# initializing training data
training = []
output_empty = [0] * len(classes)
for doc in documents:
    # initializing bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


C:\Users\HP\AppData\Local\Temp\ipykernel_27428\74795701.py:22: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [6]:
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers.schedules import ExponentialDecay

# Create model
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model
initial_learning_rate = 0.01
decay_rate = 1e-6
decay_steps = 5000  # Adjust this value based on your data and batch size

learning_rate_schedule = ExponentialDecay(
    initial_learning_rate, decay_steps=decay_steps, decay_rate=decay_rate
)

sgd = SGD(learning_rate=learning_rate_schedule, momentum=0.9, nesterov=True)

model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

# Fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("Model created.")


Epoch 1/200
13/13 [==============================] - 3s 9ms/step - loss: 3.1342 - accuracy: 0.0154
Epoch 2/200
13/13 [==============================] - 0s 7ms/step - loss: 3.0762 - accuracy: 0.0154
Epoch 3/200
13/13 [==============================] - 0s 8ms/step - loss: 3.0033 - accuracy: 0.0462
Epoch 4/200
13/13 [==============================] - 0s 6ms/step - loss: 2.9431 - accuracy: 0.1385
Epoch 5/200
13/13 [==============================] - 0s 6ms/step - loss: 2.9186 - accuracy: 0.1385
Epoch 6/200
13/13 [==============================] - 0s 5ms/step - loss: 2.8837 - accuracy: 0.1846
Epoch 7/200
13/13 [==============================] - 0s 5ms/step - loss: 2.7803 - accuracy: 0.2154
Epoch 8/200
13/13 [==============================] - 0s 5ms/step - loss: 2.7688 - accuracy: 0.1692
Epoch 9/200
13/13 [==============================] - 0s 6ms/step - loss: 2.7566 - accuracy: 0.2462
Epoch 10/200
13/13 [==============================] - 0s 5ms/step - loss: 2.7277 - accuracy: 0.2462
Epoch 11/

In [7]:
from keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random
intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

In [8]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(msg):
    ints = predict_class(msg, model)
    if not ints:
        return "I'm sorry, I don't understand. Can you please rephrase?"
    res = getResponse(ints, intents)
    return res

In [9]:
#Creating GUI with tkinter
import tkinter
from tkinter import *



# Create the root window
base = Tk()
base.title("ThoughtTherapist")
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)
base.configure(bg="#f0f0f0")

# Create a custom widget for displaying images in the chat log
class ImageLabel(Label):
    def __init__(self, master=None, **kwargs):
        super().__init__(master, **kwargs)
        self.image = kwargs.get("image")


from PIL import Image, ImageTk

# Replace "icon/chatbot.png" with the actual path to your chatbot icon image
chatbot_icon_path = "icon/chatbot.png"
desired_icon_size = (25, 25)  # Adjust this size as needed

chatbot_icon = Image.open(chatbot_icon_path)
chatbot_icon = chatbot_icon.resize(desired_icon_size, Image.ANTIALIAS)
chatbot_icon = ImageTk.PhotoImage(chatbot_icon)

# Function to display chat messages with the chatbot icon
def insert_chat_message(sender, message):
    if sender == "You":
        ChatLog.insert(END, sender + ": " + message + '\n\n', "you")
    else:
        ChatLog.window_create(END, window=ImageLabel(ChatLog, image=chatbot_icon))
        ChatLog.insert(END, " " + sender + ": " + message + '\n\n', "bot")






def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)

    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#442265", font=("Verdana", 12 ))
        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)

        # Get chatbot response
        res = chatbot_response(msg)
        
        # Display chatbot icon and response
        ChatLog.config(state=NORMAL)
        insert_chat_message("Bot", res)
        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)

#function for enter is send
# Function to handle sending when Enter key is pressed
def send_on_enter(event):
    send()









#Create Chat window
ChatLog = Text(base, bd=0, bg="#fff7d7", height="8", width="50", font=("Arial", 12))


ChatLog.config(state=DISABLED)

#Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set

#Create Button to send message
SendButton = Button(base, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#32de97", activebackground="#3c9d9b",fg='#ffffff',
                    command= send )

#Create the box to enter message
EntryBox = Text(base, bd=0, bg="white",width="29", height="5", font="Arial")
#EntryBox.bind("<Return>", send)


#Place all components on the screen
scrollbar.place(x=376,y=6, height=386)
ChatLog.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)


#line for enter is send
# Bind the <Return> key event to the send_on_enter function
EntryBox.bind("<Return>", send_on_enter)

base.mainloop()

C:\Users\HP\AppData\Local\Temp\ipykernel_27428\2999176403.py:28: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  chatbot_icon = chatbot_icon.resize(desired_icon_size, Image.ANTIALIAS)


1/1 [==============================] - 0s 62ms/step
